In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
from csv import reader
from csv import writer

In [2]:
map_names_not_same = {
                        "Alabama-Birmingham" : "UAB",
                        "Albany (NY)": "Albany",
                        "Arkansas-Pine Bluff" : "Arkansas Pine Bluff",
                        "Bethune-Cookman" : "Bethune Cookman",
                        "Bowling Green State" : "Bowling Green",
                        "Brigham Young" : "BYU",
                        "Cal State Bakersfield" : "Cal St. Bakersfield",
                        "Cal State Fullerton" : "Cal St. Fullerton",
                        "Cal State Northridge" : "Cal St. Northridge",
                        "California Baptist" : "Cal Baptist",
                        "UC-Davis" : "UC Davis",
                        "UC-Irvine" : "UC Irvine",
                        "UC-Riverside" : "UC Riverside",
                        "UCSB" : "UC Santa Barbara",
                        "University of California" : "California",
                        "Central Connecticut State" : "Central Connecticut",
                        "Central Florida" : "UCF",
                        "Citadel" : "The Citadel",
                        "College of Charleston" : "Charleston",
                        "Detroit Mercy" : "Detroit",
                        "Florida International" : "FIU",
                        "Gardner-Webb" : "Gardner Webb",
                        "Grambling" : "Grambling St.",
                        "Illinois-Chicago" : "Illinois Chicago",
                        "Purdue-Fort Wayne" : "Purdue Fort Wayne",
                        "Cal State Long Beach" : "Long Beach St.",
                        "Long Island University" : "LIU",
                        "Louisiana-Monroe" : "Louisiana Monroe",
                        "Louisiana State" : "LSU",
                        "Loyola (IL)" : "Loyola Chicago",
                        "Loyola (MD)" : "Loyola Marymount",
                        "Maryland-Baltimore County" : "UMBC",
                        "Maryland-Eastern Shore" : "Maryland Eastern Shore",
                        "Massachusetts-Lowell" : "UMass Lowell",
                        "Miami (FL)" : "Miami FL",
                        "Miami (OH)" : "Miami OH",
                        "Missouri-Kansas City" : "UMKC",
                        "Omaha" : "Nebraska Omaha",
                        "Nevada-Las Vegas" : "UNLV",
                        "North Carolina-Asheville" : "UNC Asheville",
                        "North Carolina-Greensboro" : "UNC Greensboro",
                        "North Carolina-Wilmington" : "UNC Wilmington",
                        "NC State" : "N.C. State",
                        "Pennsylvania" : "Penn",
                        "Prairie View" : "Prairie View A&M",
                        "Saint Francis (PA)" : "St. Francis PA",
                        "Saint Mary's (CA)" : "Saint Mary's",
                        "South Carolina Upstate" : "South Carolina St.",
                        "Southern California" : "USC",
                        "Southern Methodist" : "SMU",
                        "Southern Mississippi" : "Southern Miss",
                        "SIU-Edwardsville" : "SIU Edwardsville",
                        "St. Francis (NY)" : "St. Francis NY",
                        "St. John's (NY)" : "St. John's",
                        "Tennessee-Martin" : "Tennessee Martin",
                        "Texas A&M-Corpus Christi" : "Texas A&M Corpus Chris",
                        "Texas-Arlington" : "UT Arlington",
                        "Texas Christian" : "TCU",
                        "Texas-El Paso" : "UTEP",
                        "Texas-Rio Grande Valley" : "UT Rio Grande Valley",
                        "Texas-San Antonio" : "UTSA",
                        "Virginia Commonwealth" : "VCU",
                        "UConn" : "Connecticut",
                        "UMass-Lowell" : "UMass Lowell",
                        "UT-Martin" : "Tennessee Martin",
                        "UNC" : "North Carolina",
                        "Pitt" : "Pittsburgh"
                     }

In [3]:
colnames=['home','away','SCORE_1','SCORE_2','OFF_1','DEF_1','TEMPO_1','LUCK_1','OFF_2','DEF_2','TEMPO_2','LUCK_2','date']
df = pd.read_csv('all.csv', names=colnames, header=None)
df['line'] = df['SCORE_1'] - df['SCORE_2']
df['total'] = df['SCORE_1'] + df['SCORE_2']
x = df[['OFF_1','DEF_1','TEMPO_1','LUCK_1','OFF_2','DEF_2','TEMPO_2','LUCK_2']].values
y = df[['line']].values

In [4]:
from datetime import datetime
import os
def get_today():
    today = datetime.today()
    year = today.year
    month = today.month
    day = today.day
    return year,month,day
year,month,day = get_today()
path = os.getcwd() 
name = path+"\\predict\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict.csv"

In [5]:
predict = pd.read_csv(name)
predict_x = predict[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
lasso = Lasso()
lasso.fit(x, y)
predict['line'] = lasso.predict(predict_x)
predict['line'] = round(predict['line'])

In [6]:
options = webdriver.ChromeOptions() 
#options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
#"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
# "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
#options.add_argument("--start-maximized")
#self.options.add_argument("--headless")
#self.options.add_argument("--no-sandbox")
#self.options.add_argument("--disable-gpu")
options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessa
driver = webdriver.Chrome(chrome_options= options)
driver.get('https://sportsbook.draftkings.com/leagues/basketball/3230960?category=game-lines&subcategory=game')

x = True
i = 1
teams = {}
while(x):
    try:
        team_name = str(driver.find_element_by_xpath('//*[@id="root"]/section/section[2]/section/div[3]/div/div[3]/div/div/div[2]/div/div[2]/div[2]/section/div[2]/div[1]/div['+str(i)+']/a/div/div[2]/span').text)
        team_score = float(driver.find_element_by_xpath('//*[@id="root"]/section/section[2]/section/div[3]/div/div[3]/div/div/div[2]/div/div[2]/div[2]/section/div[2]/div[2]/div['+str(i)+']/div/div/div/div[1]/span').text)
        teams[team_name] = team_score 
        i += 1
    except:
        x = False

        
for key in list(teams):
    name_list = key.split()
    if name_list[-1] == "State":
        name_list[-1] = "St."
        new_name = " ".join(str(x) for x in name_list)
        home_team = new_name
        teams[home_team] = teams[key] 
        del teams[key]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [7]:
predict['draft_kings'] = predict['away'].apply(lambda x: teams[x] if x in teams else -99)
predict = predict[predict['draft_kings']!=-99]
predict['diff'] = abs(predict['line'] - predict['draft_kings'])
predict = predict.sort_values(by=['diff'], ascending = False)
predict = predict.drop(columns=['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck'])
predict

,home,away,line,draft_kings,diff
22,Liberty,South Carolina St.,30.0,24.5,5.5
7,Davidson,Charlotte,17.0,11.5,5.5
12,Stetson,Florida Atlantic,1.0,-3.0,4.0
18,Iowa St.,Kansas St.,12.0,8.0,4.0
32,Utah,Utah Valley,22.0,18.0,4.0
23,UNC Greensboro,USC Upstate,15.0,18.5,3.5
2,Illinois,Minnesota,5.0,8.0,3.0
8,Illinois St.,Chicago St.,19.0,22.0,3.0
33,VCU,Western Carolina,13.0,15.5,2.5
10,Saint Mary's,Eastern Washington,11.0,8.5,2.5


In [8]:
name = path+"\\predict_scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_scores.csv"
predict.to_csv(name, index = False)

In [9]:
def predict_scores_error(year,month,day):
    path = os.getcwd()
    predict_scores = path+"\\predict_scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_scores.csv"
    name = path+"\\predict_error\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_error.csv"
    scores = path+"\\scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    
    df = pd.read_csv(predict_scores)
    
    
    df2 = pd.read_csv(scores)
    df2['score'] = df2['home_score'] - df2['away_score']
    
    
    df['score'] = 100
    for index, row in df.iterrows():
        away = row['away']
        for index2, row2 in df2.iterrows():
            away2 = row2['away_team']
            try:
                if away2 in map_names_not_same.keys():
                    away2 = map_names_not_same[home_team]
                else:
                    name_list = away2.split()
                    if name_list[-1] == "State":
                        name_list[-1] = "St."
                        new_name = " ".join(str(x) for x in name_list)
                        away2 = new_name
                if away == away2:
                    df.loc[index,'score'] = row2['score']
            except:
                pass
    df = df[df['score']!=100]            
    return df

In [10]:
from datetime import timedelta
def get_yesterday():
    today = datetime.today()
    yesterday = today - timedelta(days = 1)
    year = yesterday.year
    month = yesterday.month
    day = yesterday.day
    return year,month,day
year,month,day = get_yesterday()

In [11]:
df = predict_scores_error(year,month,day)
name = path+"\\predict_error\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_error.csv"
df.to_csv(name, index = False)

In [18]:
#add to all 
def add_to_all(year,month,day):
    path = os.getcwd() 
    name = path+"\\predict_error\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict_error.csv"
    date = str(month)+"/"+str(day)+"/"+str(year)
    with open("all_e.csv", 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        with open(name, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    print(row[0],row[1],row[2],row[3],row[4],row[5],date)
                    csv_writer.writerow([row[0],row[1],row[2],row[3],row[4],row[5],date])

In [19]:
add_to_all(year,month,day)

Colorado St. Northern Arizona 19.0 15.5 3.5 39 2020/12/14
LSU Sam Houston St. 29.0 26.0 3.0 22 2020/12/14
George Washington William & Mary 4.0 6.5 2.5 -1 2020/12/14
San Diego Nevada -2.0 -3.5 1.5 -7 2020/12/14
Morehead St. Eastern Kentucky -5.0 -6.0 1.0 13 2020/12/14
Colorado Northern Colorado 15.0 14.5 0.5 36 2020/12/14
Creighton Marquette 7.0 7.0 0.0 -5 2020/12/14
Radford Longwood 4.0 4.0 0.0 1 2020/12/14
